<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2022/examples/blob/main/example-map-reduce/colab/hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install java first
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
#install hadoop. But we clean up first 
!rm -rf hadoop-3.3.0.tar.gz hadoop-3.3.0 /usr/local/hadoop
#download
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
#untar
!tar -xzvf hadoop-3.3.0.tar.gz >/dev/null
!mv hadoop-3.3.0 /usr/local/hadoop


In [ ]:
#get the data file
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!gunzip title.basics.tsv.gz

In [ ]:
#check path and java home
!echo $JAVA_HOME
os.environ["PATH"]+= os.pathsep + "/usr/local/hadoop/bin"
!echo $PATH

In [ ]:
#check hadoop execution and ensure its in path
!/usr/local/hadoop/bin/hadoop

The hadoop process requires a mapper file and reducer file.

In [26]:
#the magic command with two %% will take the code in cell and copy to a file.
%%file mapper.py
#!/usr/bin/python2.7
import sys,re

def map_function(title):
    fields = title.strip().split('\t')         # Split title to fields using the data delimeter
    primaryTitle = fields[2]                   # Select the required field
    WORD_RE = re.compile(r"[\w']+")
    for word in WORD_RE.findall(primaryTitle):
            yield word.strip('!"*#$&%^').strip("'").lower(), 1
    #for word in primaryTitle.strip().split():  # Split primary title by words
    #    yield word.strip('!"*#$&%^').lower(), 1                          # Use a word as a key

if __name__ == "__main__": 
  for line in sys.stdin:
      # Call map_function for each line in the input
      for key, value in map_function(line):
          # Emit key-value pairs using '|' as a delimeter  
          print(key + "|" + str(value))

Overwriting mapper.py


In [27]:
%%file reducer.py
#!/usr/bin/python2.7
import sys

prev_word = None
value_total = 0

if __name__ == "__main__": 
  for line in sys.stdin:          # For ever line in the input from stdin
      line = line.strip()         # Remove trailing characters
      word, value = line.split("|", 1)
      value = int(value)

      # Remember that Hadoop sorts mapper output by key, and the reducer takes these keys sorted
      if prev_word == word:
          value_total += value
      else:
          if prev_word != None:  # Write result to stdout
              print(prev_word + "|" + str(value_total))


          value_total = value
          prev_word = word

  if prev_word == word:  # Don't forget the last key/value pair
      print(prev_word + "|" + str(value_total))

Overwriting reducer.py


In [12]:
!head -2 title.basics.tsv

tconst	titleType	primaryTitle	originalTitle	isAdult	startYear	endYear	runtimeMinutes	genres
tt0000001	short	Carmencita	Carmencita	0	1894	\N	1	Documentary,Short


In [28]:
#give execute permission and clean output dir
!chmod +x mapper.py reducer.py
!rm -rf outputdir/

In [ ]:
#run hadoop
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input title.basics.tsv -output outputdir -mapper mapper.py -reducer reducer.py -file mapper.py -file reducer.py 

In [30]:
# The output will be created in the outputdir - see the -output flag above
!ls -l outputdir/
!mv outputdir/part-00000 outputdir/hadoop-out.txt

total 10148
-rw-r--r-- 1 root root 10388074 Feb 22 17:38 part-00000
-rw-r--r-- 1 root root        0 Feb 22 17:38 _SUCCESS


In [31]:
#download file
from google.colab import files
files.download('outputdir/hadoop-out.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>